# Занятие 3
# Прикладная алгебра и численные методы
## Сингулярное разложение (SVD), линейная регрессия
https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html#numpy.linalg.svd

In [22]:
import numpy as np
import scipy.linalg
import sympy
from sympy import latex
import matplotlib.pyplot as plt
from IPython.display import display, Latex

## Сингулярное разложение (SVD)
$$
A = Q\Sigma P^*, \quad A_{m\times n},\ Q_{m\times m}, \ \Sigma_{m\times n}, \ P_{n\times n},
$$
$Q$, $P$ - ортогональные матрицы, $\Sigma$ - диагональная, на диагонали сингулярные числа.


## Пример 1
Найти SVD
$$
\left(
\begin{matrix}
1 & 0 & 0 & 1\\
0 & 1 & 0 & 1\\
0 & 0 & 1 & 1
\end{matrix}
\right)
$$
Вначале вычислим $A^*A$:

In [5]:
A = sympy.Matrix([[1, 0, 0, 1],
                  [0, 1, 0, 1],
                  [0, 0, 1, 1]])
A_star_A = A.T * A
display(Latex(f'A^*A = {latex(A_star_A)}'))

<IPython.core.display.Latex object>

Получим собственные числа и собственные векторы с помощью eigenvects(), нормализуем векторы (чтобы норма была равна единице) методом normalized()

In [31]:
A_star_A_sympy_ev = A_star_A.eigenvects()
display(Latex(f'Собственные\ векторы\ с\ \
собственными\ числами\ {latex(A_star_A_sympy_ev)}'))
A_star_A_sympy_eigenvalues = [num for num, mult, vectors in A_star_A_sympy_ev]
A_star_A_sympy_eigenvectors = [[vector.normalized() for vector in vectors]\
                               for num, mult, vectors in A_star_A_sympy_ev]
display(Latex(f'Собственные\ числа\ {latex(A_star_A_sympy_eigenvalues)}'),
        Latex(f'Нормализованные\ собственные\ \
        векторы\ {latex(A_star_A_sympy_eigenvectors)}'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Выделим собственные векторы, обозначим их e0, e11, e12, e4, они соответствуют собственным значеним 0, 1, 1 и 4.

In [32]:
e0, e1, e4 = A_star_A_sympy_eigenvectors
e11, e12 = e1
e0, = e0
e4, = e4
display(Latex('e0 = {}, e11 = {}, e12 = {}, e4 = {}'.format(*map(latex, (e0, e11, e12, e4)))))

<IPython.core.display.Latex object>

К двум векторам, соответствующим собственному значению 1 применим процесс ортогонализации Грама-Шмидта 
$$
\begin{matrix}
e_1^{new} = e_1\\
e_2^{new} = e_2 - \frac{(e_1, e_2)}{(e_1, e_1)}e_1
\end{matrix}
$$
Полученный ортогональный вектор нормализуем, проверим ортогональность с помощью скалярного произведения:

In [33]:
e120 = e12  # копия вектора e12 для альтернативной ортогонализации
e12 = (e12 - e11.dot(e12) * e11).normalized()
P = e4.row_join(e11).row_join(e12).row_join(e0)
display(Latex(f'e11 = {latex(e11)},\ e12 = {latex(e12)},\ e11\cdot e12 = {latex(e11.dot(e12))},\ P = {latex(P)}'))

<IPython.core.display.Latex object>

Для автоматической ортогонализации с возможностью нормирования в sympy реализована функция **GramSchmidt (sympy.matrices.dense.GramSchmidt)**. Аргументы этой функции - список векторов, подлежащих ортогонализации (в виде матриц Matrix) и необязательный параметр **orthonormal** (по умолчанию False). Покажем результат работы этой функции:

In [35]:
e11, e12 = sympy.matrices.dense.GramSchmidt([e11, e120], orthonormal=True)
display(Latex(f'e120 = {latex(e120)},\ e11 = {latex(e11)},\ \
e12 = {latex(e12)},\ e11\cdot e12 = {latex(e11.dot(e12))},\ P = {latex(P)}'))

<IPython.core.display.Latex object>

Построим векторы-столбцы матрицы $Q$ и проверим, что найдено разложение SVD для исходной матрицы:

In [47]:
sigma = (2, 1, 1)
f1, f2, f3 = [A * ei / sigma[i] for i, ei in enumerate((e4, e11, e12))]
Q = f1.row_join(f2).row_join(f3)
Sig = sympy.Matrix([[2, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
display(Latex('Q = {}, Sig = {}, \
P = {}, Q  Sig  P^T =\
{}'.format(*map(latex, (Q, Sig, P, Q * Sig * P.T)))))

<IPython.core.display.Latex object>

In [48]:
display(Latex('Q = {}, Sig = {}, P = {}, Q  Sig  P^T = {}\
'.format(*[latex(item.evalf(3)) for item in (Q, Sig, P, Q * Sig * P.T)])))

<IPython.core.display.Latex object>

**Теперь то же самое, но с numpy** 

Вычислим $A^*A$:

In [50]:
A = np.array([[1, 0, 0, 1],
              [0, 1, 0, 1],
              [0, 0, 1, 1]])
A_star_A = A.T @ A
display(Latex(f'A^*A = {latex(A_star_A)}'),
        A_star_A)

<IPython.core.display.Latex object>

array([[1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 1],
       [1, 1, 1, 3]])

Найдем собственные числа и собственные векторы полученной матрицы:

In [70]:
A_star_A_eigen_vals, A_star_A_eigen_vects = np.linalg.eig(A_star_A)
print('Собственные числа \
{},\nсобственные\ векторы \n\
{}'.format(A_star_A_eigen_vals.round(2), A_star_A_eigen_vects.round(2)))

Собственные числа [ 4. -0.  1.  1.],
собственные\ векторы 
[[ 0.29  0.5  -0.82 -0.41]
 [ 0.29  0.5   0.41 -0.41]
 [ 0.29  0.5   0.41  0.82]
 [ 0.87 -0.5   0.    0.  ]]


Расположим сингулярные числа (квадратные корни из полученных собственных чисел) по убыванию, для этого сначала отсортируем их с помощью sort() по возрастанию, а затем запишем array в обратном порядке с помощью flip():

In [64]:
A_star_A_eigen_vals.sort()
A_star_A_eigen_vals_reversed = np.flip(A_star_A_eigen_vals)
display(A_star_A_eigen_vals.round(2), A_star_A_eigen_vals_reversed.round(2))

array([-0.,  1.,  1.,  4.])

array([ 4.,  1.,  1., -0.])

Обратите внимание, что .sort() изменяет array на месте, а flip() возвращает view записанного в обратном порядке array, не изменяя его.
## !!! 
По сути, мы получаем указатель на конец нашего array, так что все действия, которые мы проделаем с элементами A_star_A_eigen_vals_reversed автоматически распространятся на A_star_A_eigen_vals, поскольку это не два разных array, а один, только номера элементов считаются по-разному:

In [65]:
arr1 = np.array([1, 2, 3, 4])
arr1_reversed = np.flip(arr1)
arr1[0] = 9
display(arr1, arr1_reversed)
arr1_reversed[-2] = 8
display(arr1, arr1_reversed)

array([9, 2, 3, 4])

array([4, 3, 2, 9])

array([9, 8, 3, 4])

array([4, 3, 8, 9])

Поскольку нам достаточно работать с A_star_A_eigen_vals_reversed, не будем делать deepcopy(), чтобы сохратить в неприкосновенности A_star_A_eigen_vals.

Осталось извлечь квадратные корни из положительных элементов A_star_A_eigen_vals_reversed, и получим невозрастающую последовательность сингулярных значений.

In [66]:
sigmas = [round(np.sqrt(item), 1) for item in A_star_A_eigen_vals_reversed if item > 0]
sigmas

[2.0, 1.0, 1.0]

Составим матрицу $\Sigma$:

In [67]:
Sigma = np.hstack((np.diag(sigmas), np.zeros((3, 1))))
Sigma

array([[2., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

Обратимся к полученным вместе с собственными числами собственным векторам:

In [72]:
e4, e0, e11, e12 = [item.reshape((4, 1)) for item in  A_star_A_eigen_vects.T]
A_star_A_eigen_vects_new = [e4, e0, e11, e12]
print(*[item.round(2).T for item in A_star_A_eigen_vects], sep='\n')

[ 0.29  0.5  -0.82 -0.41]
[ 0.29  0.5   0.41 -0.41]
[0.29 0.5  0.41 0.82]
[ 0.87 -0.5   0.    0.  ]


Вычислим нормы полученных векторов и скалярное произведение 

In [76]:
print(f'Нормы {[np.linalg.norm(item).round(2) for item in A_star_A_eigen_vects_new]}, \
скалярное произведение {e11[0].dot(e12[0]).round(2)}')

Нормы [1.0, 1.0, 1.0, 1.0], скалярное произведение 0.33


Скалярное произведение не равно нулю, занчит, собственные векторы, соответствующие собственному значению 1 не ортогональны.

Нужно векторы ортогонализировать, проведем процесс ортогонализации Грама-Шмидта.

Сначала заменим e12 на вектор, ортогональный e11, затем нормализуем векторы и составим из них матрицу $P$:

In [79]:
e4, e0, e11, e12 = A_star_A_eigen_vects_new
e12 = e12 - (e11[0].dot(e12[0]) / (e11[0].dot(e11[0]))) * e11
A_star_A_eigen_vects_new[-1] = e12
e4, e0, e11, e12 = [item / np.linalg.norm(item) for item in A_star_A_eigen_vects_new]
P = np.hstack((e4, e11, e12, e0))
display(Latex(f'(e11, e12new) = {e11[0].dot(e12[0])}'),
        Latex(f'P = {latex(P.round(2))}'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Составим матрицу $Q$:

In [80]:
sigma = (2, 1, 1)
f1, f2, f3 = [A @ ei / sigma[i] for i, ei in enumerate((e4, e11, e12))]
Q = np.hstack((f1, f2, f3))
Sig = np.hstack((np.diag(sigma), np.zeros((3, 1))))
display(*[item.round(5) for item in (Q, Sig, P, Q @ Sig @ P.T)])

array([[ 0.57735, -0.8165 ,  0.     ],
       [ 0.57735,  0.40825, -0.70711],
       [ 0.57735,  0.40825,  0.70711]])

array([[2., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

array([[ 0.28868, -0.8165 ,  0.     ,  0.5    ],
       [ 0.28868,  0.40825, -0.70711,  0.5    ],
       [ 0.28868,  0.40825,  0.70711,  0.5    ],
       [ 0.86603,  0.     ,  0.     , -0.5    ]])

array([[ 1.,  0.,  0.,  1.],
       [ 0.,  1., -0.,  1.],
       [ 0., -0.,  1.,  1.]])

## Построение псевдообратной матрицы при помощи SVD
$$
A^+ = P\Sigma^+Q^*,\quad 
\Sigma^+ =
\left(
\begin{matrix}
\sigma_1^{-1} & ... & ... & ... & ... & 0\\
0 & \sigma_1^{-1} & ... & ... & ... & 0\\
0 & ... & ... & ... & ... & 0\\
0 & ... & ... & \sigma_r^{-1}  & ... & 0\\
0 & ... & ... & ... & ... & 0\\
\end{matrix}
\right)
$$

In [87]:
Sigma_plus = np.vstack((np.diag([1 / item for item in sigma]), np.zeros((1, 3))))
A_pinv_my = P @ Sigma_plus @ Q.T
display(Latex(f'A\_pinv\_my = {A_pinv_my}'),
Latex(f'np.linalg.pinv(A) = {np.linalg.pinv(A)}'),
Latex(f'\\text{{A\_pinv\_my и np.linalg.pinv(A) }}{" не " * (not np.allclose(A_pinv_my, np.linalg.pinv(A), rtol=0.1))} равны'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## И наконец SVD от numpy:

In [88]:
Q, sigma, P = np.linalg.svd(A, full_matrices=True)
Sig = np.hstack((np.diag(sigma), np.zeros((3, 1))))
display(Latex(f'P^T = {sympy.latex(P.round(2))}'), 
Latex(f'\sigma = {sympy.latex(sigma)}'),
Latex(f'Q = {sympy.latex(Q.round(2))}'), 
Latex(f'\Sigma = {sympy.latex(Sig)}'),
Latex(f'Q\Sigma P^T = {sympy.latex((Q @ Sig @ P).round(2))}'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Линейная регрессия
В некотором эксперименте измерялись значения величин $g_1$, $g_2$, $g_3$ и $H$:
$$
\begin{matrix}
g_1 & 0.12 & 0.15 & 0.9 & 0.8\\
g_2 & 2.4  & 1.8  & 3.2 & 3.6\\
g_3 & 1.1  & 1.2  & 1.3 & 1.4\\
H   & 5.1  & 6.2  & 5.5 & 4.1
\end{matrix}
$$

Найти коэффициенты $a$, $b$, $c$ линейной регрессии $H = ag_1 + bg_2 +cg_3$.

Составим матрицу $A$ столбцы которой образуют значения $g_1$, $g_2$, $g_3$.
Также составим матрицу-столбец $H$ из значений $H$,
тогда
$$
\left[\begin{matrix}a\\b\\c\end{matrix}\right] = A^+H
$$

In [91]:
A = np.array([[ 0.12, 0.15, 0.9, 0.8], 
               [ 2.4, 1.8, 3.2, 3.6],
               [1.1, 1.2, 1.3, 1.4]]).T
H1 = np.array([[5.1], [6.2], [5.5],  [4.1]])
res = np.linalg.pinv(A) @ H1
a, b, c = [round(item, 3) for item in res[:, 0]]
print(f'A = {A},\nH = {H1},\na = {a}, b = {b}, c = {c}')

A = [[0.12 2.4  1.1 ]
 [0.15 1.8  1.2 ]
 [0.9  3.2  1.3 ]
 [0.8  3.6  1.4 ]],
H = [[5.1]
 [6.2]
 [5.5]
 [4.1]],
a = -0.097, b = -1.683, c = 7.897


Вычислим относительные отклонения экспериментальных данных от функции $H = ag_1 + bg_2 + cg_3$

In [94]:
def Hfunc(g1, g2, g3):
    return a * g1 + b * g2 + c * g3
print(*[abs((Hfunc(*g) - H1[i][0])/H1[i][0]).round(3) for i, g in enumerate(A)])

0.091 0.037 0.129 0.2
